<a href="https://colab.research.google.com/github/ben0oil1/Frida-Scripts/blob/master/VITS_fast_finetuning_~_2024_01_25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 查看GPU配置
!nvidia-smi

In [ ]:
#@title STEP 1 安装运行环境
#@markdown ## 安装运行环境

!git clone https://github.com/Plachtaa/VITS-fast-fine-tuning.git
!python -m pip install --upgrade --force-reinstall regex
!python -m pip install --force-reinstall soundfile
!python -m pip install --force-reinstall gradio
!python -m pip install imageio==2.4.1
!python -m pip install --upgrade youtube-dl
!python -m pip install moviepy
%cd VITS-fast-fine-tuning

!python -m pip install --no-build-isolation -r requirements.txt
!python -m pip install --upgrade numpy
!python -m pip install --upgrade --force-reinstall numba
!python -m pip install --upgrade Cython

!python -m pip install --upgrade pyzmq
!python -m pip install pydantic==1.10.4
!python -m pip install ruamel.yaml
!python -m pip install git+https://github.com/openai/whisper.git

# build monotonic align
%cd monotonic_align/
!mkdir monotonic_align
!python setup.py build_ext --inplace
%cd ..
!mkdir pretrained_models
# download data for fine-tuning
!wget https://huggingface.co/datasets/Plachta/sampled_audio4ft/resolve/main/sampled_audio4ft_v2.zip
!unzip sampled_audio4ft_v2.zip
# create necessary directories
!mkdir video_data
!mkdir raw_audio
!mkdir denoised_audio
!mkdir custom_character_voice
!mkdir segmented_character_voice

In [ ]:
#@title STEP 1.5 选择预训练模型
#@markdown ###STEP 1.5 选择预训练模型

!wget https://huggingface.co/datasets/Plachta/sampled_audio4ft/resolve/main/VITS-Chinese/D_0.pth -O ./pretrained_models/D_0.pth
!wget https://huggingface.co/datasets/Plachta/sampled_audio4ft/resolve/main/VITS-Chinese/G_0.pth -O ./pretrained_models/G_0.pth
!wget https://huggingface.co/datasets/Plachta/sampled_audio4ft/resolve/main/VITS-Chinese/config.json -O ./configs/finetune_speaker.json

In [ ]:
#@title （可选）加载Google云端硬盘
#@markdown 加载Google云端硬盘（更快地上传数据集文件）
from google.colab import drive
drive.mount('/content/drive')

## STEP 2 上传您的角色音频数据
见[数据集上传选项](https://github.com/Plachtaa/VITS-fast-fine-tuning/blob/main/DATA.MD)  

在这里我只保留了短音频压缩包和长音频文件的两种形式；

上传方法上，一种是直接在这个界面上传，另外一种是通过Google drive上传，建议通过后者，方便数据集的复用。

###STEP 2.1 上传短音频

In [ ]:
#@markdown 上传选项1：运行该代码块会出现一个文件上传的入口
%run scripts/voice_upload.py --type zip
!unzip ./custom_character_voice/custom_character_voice.zip -d ./custom_character_voice/

In [ ]:
#@markdown 上传选项2：若您装载了Google云端硬盘，可以直接从Google云端硬盘加载文件。将`.zip`文件上传云端硬盘后，在下面填写文件路径：
ZIP_PATH = "../drive/MyDrive/samples.zip"  #@param {type:"string"}
!cp {ZIP_PATH} ./custom_character_voice/custom_character_voice.zip
!unzip ./custom_character_voice/custom_character_voice.zip -d ./custom_character_voice/

### STEP 2.2 上传长音频 （单个不应长于20分钟）

In [ ]:
#@markdown 上传选项1：

#@markdown 运行该代码块会出现一个文件上传的入口
%run scripts/voice_upload.py --type audio

In [ ]:
#@markdown 上传选项2

#@markdown 若您装载了Google云端硬盘，可以直接从Google云端硬盘加载文件。将所有长音频文件上传至云端硬盘的同一个文件夹下，在下面填写文件夹路径：
AUDIO_FOLDER_PATH = "../drive/MyDrive/long_audios/"  #@param {type:"string"}
!cp {AUDIO_FOLDER_PATH}/* ./raw_audio/

## STEP 3 自动处理所有上传的数据

In [ ]:
#@markdown 运行该单元格会对所有上传的数据进行自动去背景音&标注。

#@markdown 由于需要调用Whisper和Demucs，**运行时间可能较长。**
# 将所有视频（无论是上传的还是下载的，且必须是.mp4格式）抽取音频
%run scripts/video2audio.py
# 将所有音频（无论是上传的还是从视频抽取的，必须是.wav格式）去噪
!python scripts/denoise_audio.py
# 分割并标注长音频
!python scripts/long_audio_transcribe.py --languages "{PRETRAINED_MODEL}" --whisper_size large-v2
# 标注短音频
!python scripts/short_audio_transcribe.py --languages "{PRETRAINED_MODEL}" --whisper_size large-v2
# 底模采样率可能与辅助数据不同，需要重采样
!python scripts/resample.py

**训练质量相关：**

第一次训练建议勾选ADD_AUXILIARY

In [ ]:
#@markdown ##STEP 3.5
#@markdown 运行该单元格会生成划分好训练/测试集的最终标注，以及配置文件

#@markdown 选择是否加入辅助训练数据：
ADD_AUXILIARY = False #@param {type:"boolean"}
#@markdown 辅助训练数据是从预训练的大数据集抽样得到的，

#@markdown 作用在于防止模型在标注不准确的数据上形成错误映射。


#@markdown **以下情况请勾选：**

#@markdown - 总样本少于100条
#@markdown - 样本质量一般或较差
#@markdown - 样本来自爬取的视频

#@markdown **以下情况可以不勾选：**

#@markdown - 总样本量很大
#@markdown - 样本质量很高
#@markdown - 希望加速训练
#@markdown - 只有二次元角色

# assert(not (ADD_AUXILIARY and PRETRAINED_MODEL != "CJE")), "add auxiliary data is available only available for CJE model!"
if ADD_AUXILIARY:
  %run preprocess_v2.py --add_auxiliary_data True --languages "{PRETRAINED_MODEL}"
else:
  %run preprocess_v2.py --languages "{PRETRAINED_MODEL}"

## STEP 4 开始训练

In [ ]:
#@markdown #STEP 4 (>=20 min)
#@markdown 开始微调模型。

#@markdown 训练时长取决于你录入/上传的音频总数。

#@markdown 根据声线和样本质量的不同，所需的训练`epochs`数也不同。

#@markdown 你也可以在 `Tensorboard` 中预览合成效果，若效果满意可提前停止。
import os
os.environ['TENSORBOARD_BINARY'] = '/usr/local/bin/tensorboard'

if os.path.exists("/content/drive/MyDrive/"):
  !python scripts/rearrange_speaker.py
  !cp ./finetune_speaker.json ../drive/MyDrive/finetune_speaker.json
  !cp ./moegoe_config.json ../drive/MyDrive/moegoe_config.json

%reload_ext tensorboard
%tensorboard --logdir "./OUTPUT_MODEL"
Maximum_epochs = "200" #@param {type:"string"}
#@markdown 继续之前的模型训练
CONTINUE = True #@param {type:"boolean"}
if CONTINUE:
  !python finetune_speaker_v2.py -m "./OUTPUT_MODEL" --max_epochs "{Maximum_epochs}" --drop_speaker_embed False --cont True
else:
  !python finetune_speaker_v2.py -m "./OUTPUT_MODEL" --max_epochs "{Maximum_epochs}" --drop_speaker_embed True

In [ ]:
#@markdown ### 微调完成后，在这里尝试效果。
#@markdown 运行后会输出一个public URL, 点击进入网页版UI以使用模型
!cp ./configs/modified_finetune_speaker.json ./finetune_speaker.json
!python VC_inference.py --model_dir ./OUTPUT_MODEL/G_latest.pth --share True

# STEP 5 下载模型
## 本地部署方法请见[README](https://github.com/Plachtaa/VITS-fast-fine-tuning/blob/main/README_ZH.md)

In [ ]:
#@markdown ### 下载选项1：
#@markdown 运行该单元格，浏览器会自动下载模型和配置文件
!python scripts/rearrange_speaker.py
%run scripts/download_model.py

In [ ]:
#@markdown ### 下载选项2：
#@markdown 运行该单元格会将模型和配置文件保存到Google云端硬盘
!python scripts/rearrange_speaker.py
!cp ./G_latest.pth ../drive/MyDrive/G_latest.pth
!cp ./finetune_speaker.json ../drive/MyDrive/finetune_speaker.json
!cp ./moegoe_config.json ../drive/MyDrive/moegoe_config.json

In [ ]:
#@markdown ### 清空样本
#@markdown 运行该单元格**会清空所有已上传的样本**，需要时可使用
!rm -rf ./custom_character_voice/*
!rm -rf ./video_data/*
!rm -rf ./raw_audio/*
!rm -rf ./denoised_audio/*
!rm -rf ./segmented_character_voice/*
!rm -rf long_character_anno.txt
!rm -rf short_character_anno.txt

In [ ]:
#@markdown ### 数据备份
#@markdown 运行该单元格会将切片和标注复制到谷歌云端硬盘根目录下名为`voice_data`的文件夹下以用作其它用途
!mkdir ../drive/MyDrive/voice_data/
!cp -rf ./custom_character_voice/ ../drive/MyDrive/voice_data/custom_character_voice/
!cp -rf ./segmented_character_voice/ ../drive/MyDrive/voice_data/segmented_character_voice/
!cp long_character_anno.txt ../drive/MyDrive/voice_data/long_character_anno.txt
!cp short_character_anno.txt ../drive/MyDrive/voice_data/short_character_anno.txt